`hx173149/C3D-tensorflow`
- code review

`input_data.py`

In [1]:
import os
import tensorflow as tf
import PIL.Image as Image
import random
import numpy as np
import cv2
import time

In [2]:
for parent, dirnames, filenames in os.walk('E:/anomalydetection'):
    print(parent, dirnames, len(filenames))

E:/anomalydetection ['Anomaly-Videos-Part-1', 'Anomaly-Videos-Part-2', 'Anomaly-Videos-Part-3', 'Anomaly-Videos-Part-4', 'Normal_Videos_for_Event_Recognition', 'Testing_Normal_Videos', 'Training-Normal-Videos-Part-1', 'Training-Normal-Videos-Part-2', 'UCF_Crimes-Train-Test-Split'] 3
E:/anomalydetection\Anomaly-Videos-Part-1 ['Abuse', 'Arrest', 'Arson', 'Assault'] 0
E:/anomalydetection\Anomaly-Videos-Part-1\Abuse [] 50
E:/anomalydetection\Anomaly-Videos-Part-1\Arrest [] 50
E:/anomalydetection\Anomaly-Videos-Part-1\Arson [] 50
E:/anomalydetection\Anomaly-Videos-Part-1\Assault [] 50
E:/anomalydetection\Anomaly-Videos-Part-2 ['Burglary', 'Explosion', 'Fighting'] 0
E:/anomalydetection\Anomaly-Videos-Part-2\Burglary [] 100
E:/anomalydetection\Anomaly-Videos-Part-2\Explosion [] 50
E:/anomalydetection\Anomaly-Videos-Part-2\Fighting [] 50
E:/anomalydetection\Anomaly-Videos-Part-3 ['RoadAccidents', 'Robbery', 'Shooting'] 0
E:/anomalydetection\Anomaly-Videos-Part-3\RoadAccidents [] 150
E:/anomaly

In [3]:
def get_frames_data(filename, num_frames_per_clip=16):
    ret_arr = []
    s_index = 0
    for parent, dirnames, filenames in os.walk(filename):
        if (len(filenames) < num_frames_per_clip):
            return [], s_index
        filenames = sorted(filenames)
        s_index = random.randint(0, len(filenames) - num_frames_per_clip)
        for i in range(s_index, s_index + num_frames_per_clip):
            image_name = str(filename) + '/' + str(filenames[i])
            img = Image.open(image_name)
            img_data = np.array(img)
            ret_arr.append(img_data)
        return ret_arr, s_index

In [4]:
def read_clip_and_label(filename, batch_size, start_pos=-1, num_frames_per_clip=16, crop_size=112, shuffle=False):
    lines = open(filename, 'r')
    data, label = [], []
    batch_index, next_batch_start = 0, -1
    lines = list(lines)
    np_mean = np.load('crop_mean.npy').reshape([num_frames_per_clip, crop_size, crop_size, 3])
    # Forcing shuffle, if start_pos is not specified
    if start_pos < 0:
        shuffle = True
    if shuffle:
        video_indices = range(len(lines))
        random.seed(time.time())
        random.shuffle(video_indices)
    else:
        # Process videos sequentially
        video_indices = range(start_pos, len(lines))
    for index in video_indices:
        if (batch_index >= batch_size):
            next_batch_start = index
            break
        line = lines[index].strip('\n').split()
        dirname = line[0]
        tmp_label = line[1]
        if not shuffle:
            print("Loading a video clip from {}...".format(dirname))
        tmp_data, _ = get_frames_data(dirname, num_frames_per_clip)
        img_datas = []
        if (len(tmp_data) != 0):
            for j in xrange(len(tmp_data)):
                img = Image.fromarray(tmp_data[j].astype(np.uint8))
                if (img.width > img.height):
                    scale = float(crop_size) / float(img.height)
                    img = np.array(cv2.resize(np.array(img), (int(img.width * scale + 1), crop_size))).astype(np.float32)
                else:
                    scale = float(crop_size) / float(img.width)
                    img = np.array(cv2.resize(np.array(img), (crop_size, int(img.height * scale + 1)))).astype(np.float32)
                crop_x = int((img.shape[0] - crop_size) / 2)
                crop_y = int((img.shape[1] - crop_size) / 2)
                img = img[crop_x:crop_x+crop_size, crop_y:crop_y+crop_size,:] - np_mean[j]
                img_datas.append(img)
            data.append(img_datas)
            label.append(int(tmp_label))
            batch_index = batch_index + 1
            read_dirnames.append(dirname)

    # pad (duplicate) data/label if less than batch_size
    valid_len = len(data)
    pad_len = batch_size - valid_len
    if pad_len:
        for i in range(pad_len):
            data.append(img_datas)
            label.append(int(tmp_label))

    np_arr_data = np.array(data).astype(np.float32)
    np_arr_label = np.array(label).astype(np.int64)

    return np_arr_data, np_arr_label, next_batch_start, read_dirnames, valid_len

`c3d_model.py`

In [5]:
"""
Builds the C3D network

Implements the inference pattern for model building.
inference_c3d(): builds the model as far as si required for running the network
forward to make predictions.
"""

import tensorflow as tf

# The UCF-101 dataset has 101 clsasses
NUM_CLASSES = 101

# Images are cropped to (CROP_SIZE, CROP_SIZE)
CROP_SIZE = 112
CHANNELS = 3

# Number of frames per video clip
NUM_FRAMES_PER_CLIP = 16

In [6]:
def conv3d(name, l_input, w, b):
    return tf.nn.bias_add(
        tf.nn.conv3d(l_input, w, strides=[1, 1, 1, 1, 1], padding='SAME', name=name), 
        b)

In [7]:
def max_pool(name, l_input, k):
    return tf.nn.max_pool3d(l_input, ksize=[1, k, 2, 2, 1], strides=[1, k, 2, 2, 1], padding='SAME', name=name)

In [8]:
def inference_c3d(_X, _dropout, batch_size, _weights, _biases):
    
    # Convolution Layer
    conv1 = conv3d('conv1', _X, _weights['wc1'], _biases['bc1'])
    conv1 = tf.nn.relu(conv1, name='relu1')
    pool1 = max_pool('pool1', conv1, k=1)
    
    # Convolution Layer
    conv2 = conv3d('conv2', pool1, _weights['wc2'], _biases['bc2'])
    conv2 = tf.nn.relu(conv2, name='relu2')
    pool2 = max_pool('pool2', conv2, k=2)
    
    # Convolution Layer
    conv3 = conv3d('conv3a', pool2, _weights['wc3a'], _biases['bc3a'])
    conv3 = tf.nn.relu(conv3, name='relu3a')
    conv3 = conv3d('conv3b', conv3, _weights['wc3b'], _biases['bc3b'])
    conv3 = tf.nn.relu(conv3, name='relu3b')
    pool3 = max_pool('pool3', conv3, k=2)
    
    # Convolution Layer
    conv4 = conv3d('conv4a', pool3, _weights['wc4a'], _biases['bc4a'])
    conv4 = tf.nn.relu(conv4, name='relu4a')
    conv4 = conv3d('conv4b', conv4, _weights['wc4b'], _biases['bc4b'])
    conv4 = tf.nn.relu(conv4, name='relu4b')
    pool4 = max_pool('pool4', conv4, k=2)
    
    # Convolution Layer
    conv5 = conv3d('conv5a', pool4, _weights['wc5a'], _biases['bc5a'])
    conv5 = tf.nn.relu(conv5, name='relu5a')
    conv5 = conv3d('conv5b', conv5, _weights['wc5b'], _biases['bc5b'])
    conv5 = tf.nn.relu(conv5, name='relu5b')
    pool5 = max_pool('pool5', conv5, k=2)
    
    # Fully connected layer
    pool5 = tf.transpose(pool5, perm=[0, 1, 4, 2, 3])
    dense1 = tf.reshape(pool5, [batch_size, _weights['wd1'].get_shape().as_list()[0]]) # Reshape conv3 output to fit dense layer input
    dense1 = tf.matmul(dense1, _weights['wd1']) + _biases['bd1']
    
    dense1 = tf.nn.relu(dense1, name='fc1') # Relu activation
    dense1 = tf.nn.dropout(dense1, _dropout)
    
    dense2 = tf.nn.relu(
        tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], 
        name='fc2') # Relu activation
    dense2 = tf.nn.dropout(dense2, _dropout)
    
    # Output: class prediction
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    
    return out

`train_c3d_ucf101.py`

In [9]:
"""Trains and Evaluates the MNIST network using a feed dictionary."""
import os
import time
import numpy
import tensorflow as tf
# import input_data
# import c3d_model
import math
import numpy as np

In [10]:
# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1 # Number of my pc's gpu is 1.
flags.DEFINE_string('f', '', 'kernel')  # Jupyter notebook에서 사용시에만 필요
flags.DEFINE_integer('max_steps', 5000, 'Number of steps to run trainer.')
flags.DEFINE_integer('batch_size', 10, 'Batch size.')
FLAGS = flags.FLAGS
MOVING_AVERAGE_DECAY = 0.9999
model_save_dir = './models'

In [11]:
def placeholder_inputs(batch_size):
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                           NUM_FRAMES_PER_CLIP,
                                                           CROP_SIZE,
                                                           CROP_SIZE,
                                                           CHANNELS))
    labels_placeholder = tf.placeholder(tf.int64, shape=(batch_size))
    return images_placeholder, labels_placeholder

In [12]:
def average_gradient(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            expanded_g = tf.expand_dims(g, 0)
            grads.append(expanded_g)
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [13]:
def tower_loss(name_scope, logit, labels):
    cross_entropy_mean = tf.reduce_mean(
                    tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    tf.summary.scalar(name_scope + '_cross_entropy',
                      crosS_entropy_mean)
    # https://eyeofneedle.tistory.com/24 참고
    weight_decay_loss = tf.get_collection('weightdecay_losses')
    tf.summary.scalar(name_scope + '_weight_decay_loss', tf.reduce_mean(weight_decay_loss))
    
    # Calculate the total loss for the current tower.
    total_loss = cross_entropy_mean + weight_decay_loss
    tf.summary.scalar(name_scope + '_total_loss', tf.reduce_mean(total_loss))
    return total_loss

In [14]:
def tower_acc(logit, labels):
    correct_pred = tf.equal(tf.argmax(logit, 1), labels)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return accuracy

In [15]:
def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer)
    return var

In [16]:
def _variable_with_weight_decay(name, shape, wd):
    var = _variable_on_cpu(name, shape, tf.contrib.layers.xavier_initializer())
    if wd is not None:
        weight_decay = tf.nn.l2_loss(var) * wd
        tf.add_to_collection('weightdecay_losses', weight_decay)
    return var

In [23]:
def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    
    # Create model directory
    if not os.path.exists(model_save_dir):
        os.makedirs(model_save_dir)
    use_pertrained_model = True
    model_filename = './sports1m_finetuning_ucf101.model'
    
    with tf.Graph().as_default():
        global_step = tf.get_variable(
            'global_step',
            [],
            initializer=tf.constant_initializer(value=0),
            trainable=False)
        images_placeholder, labels_placeholder = placeholder_inputs(
                        FLAGS.batch_size + gpu_num)
        tower_grad1, tower_grad2 = [], []
        logits = []
        opt_stable = tf.train.AdamOptimizer(1e-4)
        opt_finetuning = tf.train.AdamOptimizer(1e-3)
        with tf.variable_scope('var_name') as var_scope:
            weights = {
                'wc1': _variable_with_weight_decay('wc1', [3, 3, 3, 3, 64], 0.0005),
                'wc2': _variable_with_weight_decay('wc2', [3, 3, 3, 64, 128], 0.0005),
                'wc3a': _variable_with_weight_decay('wc3a', [3, 3, 3, 128, 256], 0.0005),
                'wc3b': _variable_with_weight_decay('wc3b', [3, 3, 3, 256, 256], 0.0005),
                'wc4a': _variable_with_weight_decay('wc4a', [3, 3, 3, 256, 512], 0.0005),
                'wc4b': _variable_with_weight_decay('wc4b', [3, 3, 3, 512, 512], 0.0005),
                'wc5a': _variable_with_weight_decay('wc5a', [3, 3, 3, 512, 512], 0.0005),
                'wc5b': _variable_with_weight_decay('wc5b', [3, 3, 3, 512, 512], 0.0005),
                'wd1': _variable_with_weight_decay('wd1', [8192, 4096], 0.0005),
                'wd2': _variable_with_weight_decay('wd2', [4096, 4096], 0.0005),
                'out': _variable_with_weight_decay('wout', [4096, NUM_CLASSES], 0.0005)
            }
            biases = {
                'bc1': _variable_with_weight_decay('bc1', [64], 0.000),
                'bc2': _variable_with_weight_decay('bc2', [128], 0.000),
                'bc3a': _variable_with_weight_decay('bc3a', [256], 0.000),
                'bc3b': _variable_with_weight_decay('bc3b', [256], 0.000),
                'bc4a': _variable_with_weight_decay('bc4a', [512], 0.000),
                'bc4b': _variable_with_weight_decay('bc4b', [512], 0.000),
                'bc5a': _variable_with_weight_decay('bc5a', [512], 0.000),
                'bc5b': _variable_with_weight_decay('bc5b', [512], 0.000),
                'bd1': _variable_with_weight_decay('bd1', [4096], 0.000),
                'bd2': _variable_with_weight_decay('bd2', [4096], 0.000),
                'out': _variable_with_weight_decay('bout', [NUM_CLASSES], 0.000),
            }
        for gpu_index in range(gpu_num):
            with tf.device('/gpu:%d' % gpu_index):

                varlist2 = [weights['out'], biases['out']]
                varlist1 = list(
                    set(weights.values() + biases.values()) - set(varlist2)
                )
                logit = inference_c3d(
                    _X=images_placeholder[gpu_index * FLAGS.batch_size:(gpu_index+1) * FLAGS.batch_size],
                    _dropout=0.5,
                    batch_size=FLAGS.batch_size,
                    _weights=weights,
                    _biases=biases)
                grads1 = opt_stable.compute_gradients(loss, varlist1)
                grads2 = opt_finetuning.compute_gradients(loss, varlist2)
                tower_grads1.append(grad1)
                tower_grads2.append(grad2)
                logits.append(logit)
        logits = tf.concat(logits, axis=0)
        accuracy = tower_acc(logits, labels_placeholder)
        tf.summary.scalar('accuracy', accuracy)
        grads1 = average_gradient(tower_grads1)
        grads2 = average_gradient(tower_grads2)
        apply_gradient_op1 = opt_stable.apply_gradients(grads1)
        apply_gradient_op2 = opt_finetuning.apply_gradients(grads2, global_step=global_step)
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
        train_op = tf.group(apply_gradient_op1, apply_gradient_op2, variables_averages_op)
        null_op = tf.no_op()
                
        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver(weights.values() + biases.values())
        init = tf.global_variables_initializer()

        # Create a session for running Ops on the Graph.
        sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
        sess.run(init)
        # https://goodtogreate.tistory.com/entry/Saving-and-Restoring
        if os.path.isfile(model_filename) and use_pretrained_model:
            saver.restore(sess, model_filename)
                    
        # Create summary writter
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./visual_logs/train', sess.graph)
        test_writer = tf.summary.FileWriter('./visual_logs/test', sess.graph)
        for step in range(FLAGS.max_steps):
            start_time = time.time()
            train_images, train_labels, _, _, _ = read_clip_and_label(
                filename='list/train.list',
                batch_size=FLAGS.batch_size * gpu_num,
                num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                crop_size=CROP_SIZE,
                shuffle=True)
            sess.run(train_op,
                     feed_dict={
                         images_placeholder: train_images,
                         labels_placeholder: train_labels,
                     })
            duration = time.time() - start_time
            print('Step %d: %.3f sec' % (step, duration))
                    
            # Save a checkpoint and evaluate the model periodically.
            if ((step % 10) == 0) or ((step + 1) == FLAGS.max_steps) :
                saver.save(sess, 
                           os.path.join(
                               model_save_dir, 
                               'c3d_ucf_model'), 
                           global_step=step)
                print('Training Data Eval:')
                summary, acc = sess.run(
                    [merged, accuracy],
                    feed_dict={
                        images_placeholder: train_images,
                        labels_placeholder: train_labels,
                    })
                print('accuracy: {:.5f}'.format(acc))
                train_writer.add_summary(summary, step)
                print('Validation Data Eval:')
                val_images, val_labels, _, _, _ = read_clip_and_label(
                    filename='list/test.list',
                    batch_size=FLAGS.batch_size * gpu_num,
                    num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                    crop_size=CROP_SIZE,
                    shuffle=True)
                sumamry, acc = sess.run(
                    [merged, accuracy],
                    feed_dict={
                        images_placeholder: val_images,
                        labels_placeholder: val_labels,
                    })
                print('accuracy: {:.5f}'.format(acc))
                test_writer.add_summary(summary, step)
    print('Done')
    
def main(_):
    run_training()
    
# tf.app.run()

`predict_c3d_ucf101.py`

In [24]:
import os.path
import time
# import tensorflow as tf
# import input_data
# import c3d_model
# import numpy as np

In [27]:
# Basic model parameters as external flags.
# flags = tf.app.flags
# gpu_num = 1
# flags.DEFINE_integer('batch_size', 10, 'Batch size.')
# FLAGS = glags.FLAGS

In [29]:
def placeholder_inputs(batch_size):
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                           NUM_FRAMES_PER_CLIP,
                                                           CROP_SIZE,
                                                           CROP_SIZE,
                                                           CHANNELS))
    labels_placeholder = tf.placeholder(tf.int64, shape=(batch_size))
    return images_placeholder, labels_placeholder

In [30]:
def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer)
    return var

In [31]:
def _variable_with_weight_decay(name, shape, stddev, wd):
    var = _variable_on_cpu(name, shape, tf.truncated_normal_initializer(stddev=stddev))
    if wd is not None:
        weight_decay = tf.nn.l2_loss(var) * wd
        tf.add_to_collection('losses', weight_decay)
    return var

In [33]:
def run_test():
    model_name = "./sports1m_finetuning_ucf101.model"
    test_list_file = 'list/test.list'
    num_test_videos = len(list(open(test_list_file,'r')))
    print("Number of test videos={}".format(num_test_videos))
    
    # Get the sets of images and labels for trainig, validation, and
    images_placeholder, labels_placeholder = placeholder_inputs(FLAGS.batch_size * gpu_num)
    with tf.variable_scope('var_name') as var_scope:
        weights = {
            'wc1': _variable_with_weight_decay('wc1', [3, 3, 3, 3, 64], 0.04, 0.00),
            'wc2': _variable_with_weight_decay('wc2', [3, 3, 3, 64, 128], 0.04, 0.00),
            'wc3a': _variable_with_weight_decay('wc3a', [3, 3, 3, 128, 256], 0.04, 0.00),
            'wc3b': _variable_with_weight_decay('wc3b', [3, 3, 3, 256, 256], 0.04, 0.00),
            'wc4a': _variable_with_weight_decay('wc4a', [3, 3, 3, 256, 512], 0.04, 0.00),
            'wc4b': _variable_with_weight_decay('wc4b', [3, 3, 3, 512, 512], 0.04, 0.00),
            'wc5a': _variable_with_weight_decay('wc5a', [3, 3, 3, 512, 512], 0.04, 0.00),
            'wc5b': _variable_with_weight_decay('wc5b', [3, 3, 3, 512, 512], 0.04, 0.00),
            'wd1': _variable_with_weight_decay('wd1', [8192, 4096], 0.04, 0.001),
            'wd2': _variable_with_weight_decay('wd2', [4096, 4096], 0.04, 0.002),
            'out': _variable_with_weight_decay('wout', [4096, NUM_CLASSES], 0.04, 0.005)
        }
        biases = {
            'bc1': _variable_with_weight_decay('bc1', [64], 0.04, 0.0),
            'bc2': _variable_with_weight_decay('bc2', [128], 0.04, 0.0),
            'bc3a': _variable_with_weight_decay('bc3a', [256], 0.04, 0.0),
            'bc3b': _variable_with_weight_decay('bc3b', [256], 0.04, 0.0),
            'bc4a': _variable_with_weight_decay('bc4a', [512], 0.04, 0.0),
            'bc4b': _variable_with_weight_decay('bc4b', [512], 0.04, 0.0),
            'bc5a': _variable_with_weight_decay('bc5a', [512], 0.04, 0.0),
            'bc5b': _variable_with_weight_decay('bc5b', [512], 0.04, 0.0),
            'bd1': _variable_with_weight_decay('bd1', [4096], 0.04, 0.0),
            'bd2': _variable_with_weight_decay('bd2', [4096], 0.04, 0.0),
            'out': _variable_with_weight_decay('bout', [NUM_CLASSES], 0.04, 0.0),
        }
    logits = []
    for gpu_idnex in range(gpu_num):
        with tf.device('/gpu:%d' % gpu_index):
            logit = inference_c3d(
                images_placeholder[
                    gpu_index*FLAGS.batch_size:(gpu_index+1)*FLAGS.batch_size,
                    :,:,:,:], 
                _dropout=0.6, batch_size=FLAGS.batch_size, 
                _weights=weights, _biases=biases)
            logits.append(logit)
    logits = tf.concat(logits, axis=0)
    norm_score = tf.nn.softmax(logits)
    saver = tf.train.Saver()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    init = tf.global_variables_initializer()
    sess.run(init)
    # Create a saver for writing training checkpoints.
    saver.restore(sess, model_name)
    # And then after everything is built, start the training loop
    bufsize = 0
    write_file = open('predict_ret.txt', 'w+', bufsize)
    next_start_pos = 0
    all_steps = int((num_test_videos - 1) / (FLAGS.batch_size*gpu_num) + 1)
    for step in range(all_steps):
        # Fill a feed dictionary with the actual set of images and labels
        # for this particular training step.
        test_images, test_labels, next_start_pos, _, valid_len = \
            read_clip_and_label(test_list_file,
                                FLAGS.batch_size*gpu_num,
                                start_pos=next_start_pos)
        predict_score = norm_score.eval(
            session=sess,
            feed_dict={images_placeholder: test_images})
        for i in range(valid_len):
            true_label = test_labels[i]
            top1_predicted_label = np.argmax(predict_score[i])
            # Write results: true label, class prob for true label, predicted label, class prob for predicted label
            write_file.write('{}, {}, {}, {}\n'.format(
                true_label[0],
                predict_score[i][true_label],
                top1_predicted_label,
                predict_score[i][top1_predicted_label]))
    write_file.close()
    print('Done')
    
def main(_):
    run_test()
    
# tf.app.run()

# Using `OpenCV`, Video to Image

In [34]:
import cv2

video to image (frames) conversion

In [53]:
video_path = 'E:/UCF101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi'
vidcap = cv2.VideoCapture(video_path)

In [54]:
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
    hasFrames, image = vidcap.read()
    if hasFrames:
        cv2.imwrite('image' + str(count) + '.jpg', image) # save frame as JPG file
    return hasFrames

In [55]:
sec = 0
frameRate = 0.5 # it will capture image in each 0.5 second
count = 1
success = getFrame(sec)
while success:
    count += 1
    sec += frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [56]:
sec

7.0